In [1]:
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
import cv2
import os

In [2]:
cfg = get_cfg()
cfg.merge_from_file("./detectron2/model_zoo/configs/COCO-Detection/retinanet_R_101_FPN_3x.yaml")
cfg.MODEL.RETINANET.NUM_CLASSES = 1
cfg.DATASETS.TEST = ('flower_data/test',)
cfg.OUTPUT_DIR = 'output_flower'
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")

Loading config ./detectron2/model_zoo/configs/COCO-Detection/../Base-RetinaNet.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.
Config './detectron2/model_zoo/configs/COCO-Detection/retinanet_R_101_FPN_3x.yaml' has no VERSION. Assuming it to be compatible with latest v2.


In [3]:
import os
import numpy as np
import json
from detectron2.structures import BoxMode
import itertools
import cv2

def get_flower_dicts(csv_file, img_dir):
    df = pd.read_csv(csv_file)
    df['filename'] = df['filename'].map(lambda x: img_dir+x)

    classes = ['flower']
    #df['class'] = df['class'].map(lambda x: '')
    df['class_int'] = df['class'].map(lambda x: classes.index(x))

    dataset_dicts = []
    for filename in df['filename'].unique().tolist():
        record = {}
        
        image = cv2.imread(filename)
        
        height, width, _ = image.shape
        
        record["file_name"] = filename
        record["height"] = height
        record["width"] = width

        objs = []
        for index, row in df[(df['filename']==filename)].iterrows():
          obj= {
              'bbox': [row['xmin'], row['ymin'], row['xmax'], row['ymax']],
              'bbox_mode': BoxMode.XYXY_ABS,
              'category_id': row['class_int'],
              "iscrowd": 0
          }
          objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

In [4]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.RETINANET.NMS_THRESH_TEST = 0.9
cfg.MODEL.RETINANET.SCORE_THRESH_TEST = 0.9 
cfg.MODEL.RETINANET.NUM_CLASSES = 1 # set the testing threshold for this model
cfg.DATASETS.TEST = ('flower_data/test', )
predictor = DefaultPredictor(cfg)
tomato_metadata = MetadataCatalog.get('flowers/test')
df_test = pd.read_csv('flowers/test_labels.csv')
i = 0
dataset_dicts = get_tom_dicts('flowers/test_labels.csv', 'tomato/test/')
for d in random.sample(dataset_dicts, 10):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1], metadata=tomato_metadata, scale=0.8)
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2.imwrite('result{}.jpg'.format(i), v.get_image()[:, :, ::-1])
    i+=1

RuntimeError: CUDA error: out of memory